In [1]:
%mkdir -p /content/drive/MyDrive/GSE250378
%cd /content/drive/MyDrive/GSE250378

/content/drive/MyDrive/GSE250378


In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE250378&format=file" -O GSE250378_RAW.tar

--2024-07-24 02:04:11--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE250378&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7477585920 (7.0G) [application/x-tar]
Saving to: ‘GSE250378_RAW.tar’

GSE250378_RAW.tar   100%[===================>]   6.96G  14.4MB/s    in 9m 4s   

2024-07-24 02:13:15 (13.1 MB/s) - ‘GSE250378_RAW.tar’ saved [7477585920/7477585920]



In [ ]:
!tar -tvf GSE250378_RAW.tar | awk '{print $3/1024/1024 " MB\t" $6}'

999.375 MB	GSM7978114_D0_Lib1_10X.h5ad.gz
793.056 MB	GSM7978115_D0_Lib2_10X.h5ad.gz
812.923 MB	GSM7978116_D0_Lib3_10X.h5ad.gz
7.32938 MB	GSM7978117_D0_Lib1_pcr.h5ad.gz
4.9773 MB	GSM7978118_D0_Lib2_pcr.h5ad.gz
4.44115 MB	GSM7978119_D0_Lib3_pcr.h5ad.gz
709.734 MB	GSM7978120_D2_Lib1_10X.h5ad.gz
828.487 MB	GSM7978121_D2_Lib2_10X.h5ad.gz
689.278 MB	GSM7978122_D2_Lib3_10X.h5ad.gz
4.689 MB	GSM7978123_D2_Lib1_pcr.h5ad.gz
7.10901 MB	GSM7978124_D2_Lib2_pcr.h5ad.gz
4.44308 MB	GSM7978125_D2_Lib3_pcr.h5ad.gz
753.288 MB	GSM7978126_D4_Lib1_10X.h5ad.gz
730.535 MB	GSM7978127_D4_Lib2_10X.h5ad.gz
767.445 MB	GSM7978128_D4_Lib3_10X.h5ad.gz
5.64311 MB	GSM7978129_D4_Lib1_pcr.h5ad.gz
4.25694 MB	GSM7978130_D4_Lib2_pcr.h5ad.gz
4.15263 MB	GSM7978131_D4_Lib3_pcr.h5ad.gz


In [ ]:
!tar -xvf GSE250378_RAW.tar
!gzip -d *.gz

GSM7978114_D0_Lib1_10X.h5ad.gz
GSM7978115_D0_Lib2_10X.h5ad.gz
GSM7978116_D0_Lib3_10X.h5ad.gz
GSM7978117_D0_Lib1_pcr.h5ad.gz
GSM7978118_D0_Lib2_pcr.h5ad.gz
GSM7978119_D0_Lib3_pcr.h5ad.gz
GSM7978120_D2_Lib1_10X.h5ad.gz
GSM7978121_D2_Lib2_10X.h5ad.gz
GSM7978122_D2_Lib3_10X.h5ad.gz
GSM7978123_D2_Lib1_pcr.h5ad.gz
GSM7978124_D2_Lib2_pcr.h5ad.gz
GSM7978125_D2_Lib3_pcr.h5ad.gz
GSM7978126_D4_Lib1_10X.h5ad.gz
GSM7978127_D4_Lib2_10X.h5ad.gz
GSM7978128_D4_Lib3_10X.h5ad.gz
GSM7978129_D4_Lib1_pcr.h5ad.gz
GSM7978130_D4_Lib2_pcr.h5ad.gz
GSM7978131_D4_Lib3_pcr.h5ad.gz


In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
!. "$HOME/.cargo/env"

In [8]:
%ls

genes.tsv.gz                 GSM7978118_D0_Lib2_pcr.h5ad  GSM7978125_D2_Lib3_pcr.h5ad
GSE250378.ipynb              GSM7978119_D0_Lib3_pcr.h5ad  GSM7978126_D4_Lib1_10X.h5ad
GSE250378_RAW.tar            GSM7978120_D2_Lib1_10X.h5ad  GSM7978127_D4_Lib2_10X.h5ad
GSM7978114_D0_Lib1_10X.h5ad  GSM7978121_D2_Lib2_10X.h5ad  GSM7978128_D4_Lib3_10X.h5ad
GSM7978115_D0_Lib2_10X.h5ad  GSM7978122_D2_Lib3_10X.h5ad  GSM7978129_D4_Lib1_pcr.h5ad
GSM7978116_D0_Lib3_10X.h5ad  GSM7978123_D2_Lib1_pcr.h5ad  GSM7978130_D4_Lib2_pcr.h5ad
GSM7978117_D0_Lib1_pcr.h5ad  GSM7978124_D2_Lib2_pcr.h5ad  GSM7978131_D4_Lib3_pcr.h5ad


In [2]:
%pip install scanpy python-dotenv geomux -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from geomux import Geomux, read_anndata
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

In [11]:
file_list = [file for file in os.listdir() if file.startswith('GSM')]
gex_list = [file for file in file_list if '10X' in file]
crop_list = [file for file in file_list if 'pcr' in file]
sample_list = ["_".join(file.split("_")[1:3]) for file in gex_list]
pair_list = []
for sample in sample_list:
    for file1 in gex_list:
        if sample in file1:
            gex = file1
    for file2 in crop_list:
        if sample in file2:
            crop = file2
    pair_list.append((gex, crop))

In [12]:
sample_list

['D0_Lib1',
 'D0_Lib2',
 'D0_Lib3',
 'D2_Lib1',
 'D2_Lib2',
 'D2_Lib3',
 'D4_Lib1',
 'D4_Lib2',
 'D4_Lib3']

In [13]:
pair_list

[('GSM7978114_D0_Lib1_10X.h5ad', 'GSM7978117_D0_Lib1_pcr.h5ad'),
 ('GSM7978115_D0_Lib2_10X.h5ad', 'GSM7978118_D0_Lib2_pcr.h5ad'),
 ('GSM7978116_D0_Lib3_10X.h5ad', 'GSM7978119_D0_Lib3_pcr.h5ad'),
 ('GSM7978120_D2_Lib1_10X.h5ad', 'GSM7978123_D2_Lib1_pcr.h5ad'),
 ('GSM7978121_D2_Lib2_10X.h5ad', 'GSM7978124_D2_Lib2_pcr.h5ad'),
 ('GSM7978122_D2_Lib3_10X.h5ad', 'GSM7978125_D2_Lib3_pcr.h5ad'),
 ('GSM7978126_D4_Lib1_10X.h5ad', 'GSM7978129_D4_Lib1_pcr.h5ad'),
 ('GSM7978127_D4_Lib2_10X.h5ad', 'GSM7978130_D4_Lib2_pcr.h5ad'),
 ('GSM7978128_D4_Lib3_10X.h5ad', 'GSM7978131_D4_Lib3_pcr.h5ad')]

In [6]:
def assign_guides_conditions(crop, adata):
    matrix = read_anndata(crop)
    matrix = matrix.loc[adata.obs_names]
    gx = Geomux(
        matrix,
        cell_names=matrix.index.values,
        guide_names=matrix.columns.values,
    )
    gx.test()
    assignments = gx.assignments(pvalue_threshold=0.05, lor_threshold=1.)
    assignments.dropna(inplace=True)
    assignments = assignments.loc[
        (assignments['n_umi'] > 0) & (assignments['assignment'].apply(lambda x: len(x) > 0))
    ]
    assignments['perturbation_name'] = assignments['assignment'].apply(
        lambda x: "+".join(x)
    )
    assignments['condition'] = assignments['assignment'].apply(
        lambda x: 'non-targeting' if all([item.startswith('NT') for item in x]) else 'targeting'
    )
    return assignments.set_index('cell_id')

In [14]:
adatas = {}
for (gex, crop), sample in zip(pair_list, sample_list):
    print(f"Processing {sample}")
    adata_gex = ad.read_h5ad(gex)
    adata_gex.var['gene_id'] = adata_gex.var.index
    adata_gex.var['gene_name'] = adata_gex.var['gene_name'].astype(str)
    adata_gex = adata_gex[:, adata_gex.var['gene_name'] != '']
    adata_gex.var.set_index('gene_name', inplace=True)
    adata_gex.var_names_make_unique()

    adata_crop = ad.read_h5ad(crop)
    adata_crop.var.index.name = 'gene_name'

    shared_bc = adata_gex.obs.index.intersection(adata_crop.obs.index)
    adata_gex = adata_gex[shared_bc]
    adata_crop = adata_crop[shared_bc]

    adata = ad.concat([adata_gex, adata_crop], axis=1)
    adata.var_names_make_unique()
    obs_guides = assign_guides_conditions(crop, adata_crop)
    adata.obs[obs_guides.columns] = obs_guides
    adata = adata[adata.obs.dropna().index]

    adata.obs_names_make_unique()
    adatas[sample] = adata
    print(f"{sample} done | {adata.shape}")

adata = ad.concat(adatas, merge='unique', join='outer', label='sample_id')
adata.var_names_make_unique()
adata

Processing D0_Lib1
D0_Lib1 done | (10349, 40930)
Processing D0_Lib2
D0_Lib2 done | (18311, 40930)
Processing D0_Lib3
D0_Lib3 done | (18731, 40930)
Processing D2_Lib1
D2_Lib1 done | (16834, 40930)
Processing D2_Lib2
D2_Lib2 done | (11082, 40930)
Processing D2_Lib3
D2_Lib3 done | (16169, 40930)
Processing D4_Lib1
D4_Lib1 done | (16534, 40930)
Processing D4_Lib2
D4_Lib2 done | (16565, 40930)
Processing D4_Lib3
D4_Lib3 done | (16715, 40930)


AnnData object with n_obs × n_vars = 141290 × 40930
    obs: 'assignment', 'counts', 'pvalues', 'log_odds', 'moi', 'n_umi', 'min_pvalue', 'max_count', 'tested', 'perturbation_name', 'condition', 'sample_id'

In [15]:
adata.obs.head()

,assignment,counts,pvalues,log_odds,moi,n_umi,min_pvalue,max_count,tested,perturbation_name,condition,sample_id
barcode,,,,,,,,,,,,
AAACCCAAGACAGCGT,[NTC_3083],[8.0],[0.006202303351983454],[inf],1.0,40.0,6.202303e-03,8.0,True,NTC_3083,non-targeting,D0_Lib1
AAACCCAAGGCATGCA,[SYNGAP1],[14.0],[2.273813813837484e-05],[inf],1.0,220.0,2.273814e-05,14.0,True,SYNGAP1,targeting,D0_Lib1
AAACCCACAACAGATA,[DNMT3A],[30.0],[2.1387979227353947e-25],[inf],1.0,112.0,2.138798e-25,30.0,True,DNMT3A,targeting,D0_Lib1
AAACCCACAATGCTCA,[KIAA0232],[19.0],[6.801615038212467e-15],[31.905217857524345],1.0,102.0,6.801615e-15,19.0,True,KIAA0232,targeting,D0_Lib1
AAACCCAGTACCAGAG,[TBL1XR1],[18.0],[7.547488836690469e-07],[inf],1.0,193.0,7.547489e-07,18.0,True,TBL1XR1,targeting,D0_Lib1


In [16]:
adata.obs['time_points'] = adata.obs['sample_id'].str.split("_").str[0]
adata.obs['GEM_library'] = adata.obs['sample_id'].str.split("_").str[1]
adata.obs['organism'] = ORGANISMS[0]
adata.obs['crispr_type'] = 'CRISPRi'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'neural progenitor cells and neurons'

In [17]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [20]:
for col in ['assignment', 'counts', 'pvalues', 'log_odds']:
    adata.obs[col] = adata.obs[col].apply(lambda x: "|".join([str(i) for i in x]))

In [29]:
adata.obs.drop(columns=['assignment', 'tested'], inplace=True)
adata.obs.head()

,counts,pvalues,log_odds,moi,n_umi,min_pvalue,max_count,perturbation_name,condition,sample_id,time_points,GEM_library,organism,crispr_type,cancer_type,cell_type
barcode,,,,,,,,,,,,,,,,
AAACCCAAGACAGCGT,8.0,0.006202303351983454,inf,1.0,40.0,6.202303e-03,8.0,NTC_3083,non-targeting,D0_Lib1,D0,Lib1,Humans (Homo sapiens),CRISPRi,non-cancer,neural progenitor cells and neurons
AAACCCAAGGCATGCA,14.0,2.273813813837484e-05,inf,1.0,220.0,2.273814e-05,14.0,SYNGAP1,targeting,D0_Lib1,D0,Lib1,Humans (Homo sapiens),CRISPRi,non-cancer,neural progenitor cells and neurons
AAACCCACAACAGATA,30.0,2.1387979227353947e-25,inf,1.0,112.0,2.138798e-25,30.0,DNMT3A,targeting,D0_Lib1,D0,Lib1,Humans (Homo sapiens),CRISPRi,non-cancer,neural progenitor cells and neurons
AAACCCACAATGCTCA,19.0,6.801615038212467e-15,31.905217857524345,1.0,102.0,6.801615e-15,19.0,KIAA0232,targeting,D0_Lib1,D0,Lib1,Humans (Homo sapiens),CRISPRi,non-cancer,neural progenitor cells and neurons
AAACCCAGTACCAGAG,18.0,7.547488836690469e-07,inf,1.0,193.0,7.547489e-07,18.0,TBL1XR1,targeting,D0_Lib1,D0,Lib1,Humans (Homo sapiens),CRISPRi,non-cancer,neural progenitor cells and neurons


In [30]:
adata.write_h5ad("./GSE250378_merged.h5ad")